### Data Preperation

In [35]:
from gensim.models.word2vec import Word2Vec
import pickle 
import numpy as np
import random
import string
from keras.preprocessing.sequence import pad_sequences
import gensim.downloader
import pandas as pd

from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [ ]:
word2vec_vectors = gensim.downloader.load('word2vec-google-news-300')
data=pd.read_csv('ner.csv', encoding = "ISO-8859-1", error_bad_lines=False)

In [ ]:
data=data.drop(['Unnamed: 0', 'lemma', 'next-lemma', 'next-next-lemma', 'next-next-pos',
       'next-next-shape', 'next-next-word', 'next-pos', 'next-shape',
       'next-word', 'prev-iob', 'prev-lemma', 'prev-pos',
       'prev-prev-iob', 'prev-prev-lemma', 'prev-prev-pos', 'prev-prev-shape',
       'prev-prev-word', 'prev-shape', 'prev-word',"pos",'shape'],axis=1)
data=data.dropna()

In [ ]:
data['tag'].value_counts().plot.barh()

In [ ]:
def get_dict_map(data, token_or_tag):
    ''''
    generate index for token and tags
    for feeding into the network
    
    ''''
    tok2idx = {}
    idx2tok = {}
    
    if token_or_tag == 'token':
        vocab = list(set(data['word'].to_list()))
    else:
        vocab = list(set(data['tag'].to_list()))
    
    idx2tok = {idx:tok for  idx, tok in enumerate(vocab)}
    tok2idx = {tok:idx for  idx, tok in enumerate(vocab)}
    return tok2idx, idx2tok
    
def data_group():
    data['word_idx'] = data['word'].map(token2idx)
    data['tag_idx'] = data['tag'].map(tag2idx)
    data_group = data.groupby(['sentence_idx'],as_index=False
                               )['word','tag','word_idx','tag_idx'].agg(lambda x: list(x))
    return data_group

def getitem():
    ''''
    in case the word is out of vocabulary, i use random vector to fit
    '''
    new_vec = np.array([random.random() for i in range(300)])
    vec.append(new_vec)
    return new_vec

def matrix(model):
    embedding_matrix1=np.zeros((len(token2idx) + 3,300))
    for word, v in token2idx.items():
        if word in model.wv.vocab:
            embedding_matrix1[v]=model[word]
        else:
            embedding_matrix1[v]=getitem()
            
    embedding_matrix1[30174]=getitem()      
    return embedding_matrix1

In [ ]:

def get_pad_train_test_val(data_group, data):
    '''
    Pad data to the max length
    for token padding, we all pad the last token, 
    and tag, we pad it to 'O'
    '''
    n_token = len(list(set(data['word'].to_list())))
    n_tag = len(list(set(data['tag'].to_list())))
    
    tokens = data_group['word_idx'].tolist()
    maxlen = max([len(s) for s in tokens])
    pad_tokens = pad_sequences(tokens, maxlen=maxlen, dtype='int32', padding='post', value=n_token-1)
    
    tags = data_group['tag_idx'].tolist()
    pad_tags = pad_sequences(tags, maxlen=maxlen, dtype='int32', padding='post', value=tag2idx["O"])
    
    n_tags = len(tag2idx)
    pad_tags = [to_categorical(i, num_classes=n_tags) for i in pad_tags]
    
    
    train_tokens, test_tokens, train_tags, test_tags = train_test_split(pad_tokens, pad_tags, test_size=0.1, train_size=0.9, random_state=2020)

    print(
        'train_tokens length:', len(train_tokens),
        '\ntest_tokens length:', len(test_tokens),
        '\ntrain_tags:', len(train_tags),
        '\ntest_tags:', len(test_tags)
    )
    
    return train_tokens, test_tokens, train_tags, test_tags


### Model

In [1]:
import pickle

In [3]:
train_tags=pickle.load(open('train_tags.pickle','rb'))

In [4]:
train_tokens=pickle.load(open('train_tokens.pickle','rb'))

In [5]:
test_tokens=pickle.load(open('test_tokens.pickle','rb'))

In [101]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Embedding, Bidirectional,Dense
from crf2 import CRF
# from CRF import CRF

class MyBiLSTMCRF:
    def __init__(self, vocabSize, maxLen,tagSum,vecSize=300,learning_rate=0.01):
        self.vocabSize = vocabSize
        self.vecSize = vecSize
        self.maxLen = maxLen
        self.tagSum = tagSum
        self.learning_rate=learning_rate
        self.buildBiLSTMCRF()

    def buildBiLSTMCRF(self):
        myModel=Sequential()
        myModel.add(Input(shape=(self.maxLen,)))
        myModel.add(Embedding(self.vocabSize, self.vecSize,weights=[embedding_weight]))
        myModel.add(Bidirectional(tf.keras.layers.LSTM(
                    self.tagSum, return_sequences=True), merge_mode='concat'))
        myModel.add(Bidirectional(tf.keras.layers.LSTM(
                    self.tagSum, return_sequences=True), merge_mode='concat'))
        myModel.add(Dense(self.tagSum))
        crf=CRF(self.tagSum,name='crf_layer')
        myModel.add(crf)
        myModel.compile(Adam(learning_rate=self.learning_rate),loss={'crf_layer': crf.get_loss},accuracy='')
        self.myBiLSTMCRF=myModel
        
    def fit(self,X,y,epochs=100,batch_size=256, verbose=1, validation_split=0.2,transParam=None,):
        if len(y.shape)==3:
            y=np.argmax(y,axis=-1)
        history=self.myBiLSTMCRF.fit(X,y,epochs=epochs,batch_size=batch_size, verbose=verbose, validation_split=validation_split)

        return history

    def predict(self,X):
        preYArr=self.myBiLSTMCRF.predict(X)
        return preYArr

    def save(self):
        save=self.myBiLSTMCRF.save('bi-lstm-crf')
        return save

In [56]:
myModel=MyBiLSTMCRF(30175,140, 17)

In [52]:
myModel.myBiLSTMCRF.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 140, 300)          9052500   
_________________________________________________________________
bidirectional_14 (Bidirectio (None, 140, 34)           43248     
_________________________________________________________________
bidirectional_15 (Bidirectio (None, 140, 34)           7072      
_________________________________________________________________
dense_5 (Dense)              (None, 140, 17)           595       
_________________________________________________________________
crf_layer (CRF)              (None, 140)               629       
Total params: 9,104,044
Trainable params: 9,104,044
Non-trainable params: 0
_________________________________________________________________


In [26]:
train_tags=pickle.load(open('train_tags.pickle','rb'))
train_tokens=pickle.load(open('train_tokens.pickle','rb'))

In [57]:
myModel.fit(train_tokens,np.array(train_tags))

Epoch 1/100
99/99 [==============================] - 92s 930ms/step - loss: 26.9687 - val_loss: 9.2687
Epoch 2/100
99/99 [==============================] - 112s 1s/step - loss: 6.0339 - val_loss: 4.2955
Epoch 3/100
99/99 [==============================] - 113s 1s/step - loss: 2.9916 - val_loss: 3.4455
Epoch 4/100
99/99 [==============================] - 115s 1s/step - loss: 2.1633 - val_loss: 3.1659
Epoch 5/100
99/99 [==============================] - 114s 1s/step - loss: 1.7045 - val_loss: 3.1330
Epoch 6/100
99/99 [==============================] - 115s 1s/step - loss: 1.3737 - val_loss: 3.2210
Epoch 7/100
99/99 [==============================] - 117s 1s/step - loss: 1.1250 - val_loss: 3.3609
Epoch 8/100
99/99 [==============================] - 119s 1s/step - loss: 0.9138 - val_loss: 3.5401
Epoch 9/100
99/99 [==============================] - 116s 1s/step - loss: 0.7460 - val_loss: 3.8093
Epoch 10/100
99/99 [==============================] - 119s 1s/step - loss: 0.5984 - val_loss: 4.0

### Metrics

In [58]:
from seqeval.metrics import classification_report,precision_score, recall_score, f1_score

In [59]:
tag2idx=pickle.load(open('tag2idx.pickle','rb'))

In [62]:
test_tokens=pickle.load(open('test_tokens.pickle','rb'))

In [63]:
test_pred=myModel.predict(test_tokens)

In [74]:
from keras.utils import to_categorical
idx2tag={}
for i,v in tag2idx.items():
    idx2tag[v]=i
    
ct=[]
for i,v in idx2tag.items():
    ct.append(list(to_categorical(i, num_classes=18)))
tags_nd=dict(zip(idx2tag.keys(),ct))

In [67]:
test_tags=pickle.load(open('test_tags.pickle','rb'))

In [88]:
idx2tag

{0: 'B-eve',
 1: 'I-per',
 2: 'I-org',
 3: 'B-geo',
 4: 'B-art',
 5: 'B-tim',
 6: 'I-gpe',
 7: 'O',
 8: 'I-geo',
 9: 'B-org',
 10: 'I-art',
 11: 'B-per',
 12: 'B-nat',
 13: 'I-tim',
 14: 'B-gpe',
 15: 'I-nat',
 16: 'I-eve'}

In [95]:
def truelabel(pred):
    out = []
    for i in pred:
        out_i = []
        for p in i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i])
        out.append(out_i)
    return out

def testlabel(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            for k,v in tags_nd.items():
                if k==p:
                    m=idx2tag[k]
                    out_i.append(m)
        out.append(out_i)
    return out
    
true_labels = truelabel(test_tags)
test_labels = testlabel(test_pred)


In [97]:
print(classification_report(true_labels,test_labels))

              precision    recall  f1-score   support

         art       0.21      0.17      0.19        30
         eve       0.29      0.28      0.29        32
         geo       0.84      0.83      0.83      3829
         gpe       0.93      0.90      0.91      1712
         nat       0.10      0.10      0.10        21
         org       0.61      0.64      0.63      2004
         per       0.72      0.71      0.72      1698
         tim       0.84      0.82      0.83      2016

   micro avg       0.79      0.78      0.79     11342
   macro avg       0.57      0.56      0.56     11342
weighted avg       0.79      0.78      0.79     11342

